# Modelos Redes Neurais
## Cabeçalho
Criado por: Danilo Steckelberg <br>
Criado para: Trabalho de conclusão de curso IBM Deep Learning (https://www.coursera.org/learn/deep-learning-reinforcement-learning) <br>
Criado em: 2023-01-19 <br>
Direitos dos dados: Evi Brasil <br>

## Carregar dados
Origem dos dados: Rotina em R [File](file://C:\\Users\\danil\\Documents\\Danilo\\Evi%20Data%20Pipeline\\evi_dados\\tcc_mba_dsa\\papers\\OUT_comparacao_modelos.R)

Tratamento dos dados realizado nas rotinas em R

In [2]:
import pandas as pd
import numpy as np
import itertools
dados_entrada = pd.read_csv('.\\data\\nn_input.csv')

In [3]:
import tensorflow as tf
tf.random.set_seed(42)

## Optimization function

In [4]:
def get_models(num_layers: int,
    min_nodes_per_layer: int,
    max_nodes_per_layer: int,
    node_step_size: int,
    input_shape: tuple,
    hidden_layer_activation: str = 'relu',
    num_nodes_at_output: int = 1,
    output_layer_activation: str = 'sigmoid') -> list:
    node_options = list(range(min_nodes_per_layer, max_nodes_per_layer + 1, node_step_size))
    layer_possibilities = [node_options] * num_layers
    layer_node_permutations = list(itertools.product(*layer_possibilities))

    models = []
    for permutation in layer_node_permutations:
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape = input_shape))
        model_name = ''

        for nodes_at_layer in permutation:
            model.add(tf.keras.layers.Dense(nodes_at_layer, activation = hidden_layer_activation))
            model_name += f'dense{nodes_at_layer}_'

        model.add(tf.keras.layers.Dense(num_nodes_at_output, activation = output_layer_activation))
        model._name = model_name[:-1]
        models.append(model)
    
    return models

In [5]:
all_models = get_models(
    num_layers=4,
    min_nodes_per_layer=5,
    max_nodes_per_layer=20,
    node_step_size=5,
    input_shape=(24,),
    hidden_layer_activation='relu',
    output_layer_activation='linear'
)

In [6]:
print(f'# of models: {len(all_models)}')

# of models: 256


## Training Function


In [7]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
def optimize(models: list,
             X_train: np.array,
             y_train: np.array,
             X_test: np.array,
             y_test: np.array,
             epochs: int = 50,
             verbose: int = 0) -> pd.DataFrame:
    
    results = []

    def train(model: tf.keras.Sequential) -> dict:
        model.compile(
            loss=tf.keras.losses.mean_squared_error,
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[tf.keras.metrics.mean_squared_error]
        )
        model.fit(X_train, y_train, epochs = epochs, verbose = verbose)
        preds = model.predict(X_test)

        return {
            'model_name': model.name,
            'mse': mean_squared_error(y_test, preds),
            'r_square': r2_score(y_test, preds),
        }
    
    i = 0
    for model in models:
        i += 1
        print(f'{i} out of {len(models)}')
        try:
            print(model.name, end='...')
            res = train(model=model)
            results.append(res)
        except Exception as e:
            print(f'{model.name} --> {str(e)}')

    return pd.DataFrame(results)

In [9]:
dados_entrada.keys()
df = dados_entrada.copy(deep=True)
df = df.dropna()

# df['RoasHigh'] = [1 if yvar >= 0.2 else 0 for yvar in df['yvar']]

df['RoasHigh'] = df['yvar']

df.drop(['views_5s_x_days', 'yvar'], axis = 1, inplace=True)

In [10]:
from sklearn.model_selection import train_test_split

X = df.drop('RoasHigh', axis = 1)
y = df['RoasHigh']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
%%time

optimization_results = optimize(
    models = all_models,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test
)

1 out of 256
5/5 [==============================] - 0s 5ms/step
2 out of 256
5/5 [==============================] - 0s 1ms/step
3 out of 256
5/5 [==============================] - 0s 1ms/step
4 out of 256
5/5 [==============================] - 0s 2ms/step
5 out of 256
5/5 [==============================] - 0s 1ms/step
6 out of 256
5/5 [==============================] - 0s 1ms/step
7 out of 256
5/5 [==============================] - 0s 2ms/step
8 out of 256
5/5 [==============================] - 0s 1ms/step
9 out of 256
5/5 [==============================] - 0s 2ms/step
10 out of 256
5/5 [==============================] - 0s 2ms/step
11 out of 256
5/5 [==============================] - 0s 3ms/step
12 out of 256
5/5 [==============================] - 0s 2ms/step
13 out of 256
5/5 [==============================] - 0s 2ms/step
14 out of 256
5/5 [==============================] - 0s 2ms/step
15 out of 256
5/5 [==============================] - 0s 2ms/step
16 out of 256
5/5 [===============

In [13]:
optimization_results.sort_values(by = 'r_square', ascending=False)

,model_name,mse,r_square
73,dense10_dense5_dense15_dense10,0.005824,0.503960
1,dense5_dense5_dense5_dense10,0.006105,0.480021
20,dense5_dense10_dense10_dense5,0.006133,0.477674
197,dense20_dense5_dense10_dense10,0.006213,0.470859
14,dense5_dense5_dense20_dense15,0.006291,0.464195
...,...,...,...
183,dense15_dense20_dense10_dense20,0.009605,0.181923
3,dense5_dense5_dense5_dense20,0.009727,0.171533
118,dense10_dense20_dense10_dense15,0.009878,0.158693
60,dense5_dense20_dense20_dense5,0.010035,0.145343


In [14]:
optimization_results.to_csv('optimization_results_4layers_relu.csv')

In [60]:
def get_model_parameters(model) -> dict:
    trainableParams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
    totParams = trainableParams + nonTrainableParams
    return {
        'model_name':model.name,
        'params':totParams
    }

all_models = get_models(
    num_layers=4,
    min_nodes_per_layer=5,
    max_nodes_per_layer=20,
    node_step_size=5,
    input_shape=(24,),
    hidden_layer_activation='relu',
    output_layer_activation='linear'
)


prs = []
for model in all_models:
    pr = get_model_parameters(model)
    prs.append(pr)

print(pd.DataFrame(prs))

pd.DataFrame(prs).to_csv('4layer_params.csv')

                 model_name  params
0      dense5_dense5_dense5   191.0
1     dense5_dense5_dense10   226.0
2     dense5_dense5_dense15   261.0
3     dense5_dense5_dense20   296.0
4     dense5_dense10_dense5   246.0
..                      ...     ...
59  dense20_dense15_dense20  1156.0
60   dense20_dense20_dense5  1031.0
61  dense20_dense20_dense10  1141.0
62  dense20_dense20_dense15  1251.0
63  dense20_dense20_dense20  1361.0

[64 rows x 2 columns]


In [53]:
print(d)

Empty DataFrame
Columns: [model_name, params]
Index: []
